In [ ]:
# default_exp pandas

# Pandas

> Utilities for pandas

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import pandas as pd
import pandas_flavor
from datetime import datetime 
import typing as T

In [ ]:
import numpy as np
from dessiccate.plotting import set_plt_defaults
import time
set_plt_defaults()

## `DataFrame` methods

### `ends`

This function shows combines the `.head()` and `.tail()` method of the `pd.DataFrame`.

Options:

* `n`: number of rows to show for the head and tail. The resulting df will have 2\*n rows, unless the df has <2\*n rows.

In [ ]:
# export
@pandas_flavor.register_dataframe_method
@pandas_flavor.register_series_method
def ends(self, n:int=3)-> pd.DataFrame:
    """
    Combines the .head and .tail methods to show both ends
    of a pd.DataFrame.
    
    Options:
    * n: number of rows to show for the head and tail. The resulting
         DataFrame will have 2*n rows, unless the df has <2*n rows.
    """
    return pd.concat([self.head(n), self.tail(n)], axis=0).drop_duplicates()

In [ ]:
df = pd.DataFrame(np.random.rand(20, 5))

In [ ]:
df.head()

,0,1,2,3,4
0,0.938650,0.170227,0.388050,0.939878,0.966589
1,0.155440,0.026628,0.920497,0.724409,0.006836
2,0.706318,0.242996,0.113334,0.950816,0.036472
3,0.949107,0.227721,0.076558,0.070814,0.173940
4,0.409216,0.928254,0.144014,0.351909,0.674854


In [ ]:
df.tail()

,0,1,2,3,4
15,0.216234,0.457282,0.103442,0.357210,0.288601
16,0.435056,0.211597,0.185679,0.824459,0.569631
17,0.119716,0.050019,0.530540,0.291529,0.231619
18,0.795463,0.211936,0.643802,0.306037,0.934522
19,0.518866,0.011515,0.941467,0.886537,0.237553


In [ ]:
df.ends()

,0,1,2,3,4
0,0.938650,0.170227,0.388050,0.939878,0.966589
1,0.155440,0.026628,0.920497,0.724409,0.006836
2,0.706318,0.242996,0.113334,0.950816,0.036472
17,0.119716,0.050019,0.530540,0.291529,0.231619
18,0.795463,0.211936,0.643802,0.306037,0.934522
19,0.518866,0.011515,0.941467,0.886537,0.237553


### `group_by_summary`

This method returns a df with the mean, median, sem, std, and count of each column.

Options:

* `by`: Colums to group by. These will be the index of the df returned.
* `filter_cols`: if `None`, will use all the columns. Otherwise, pass column name or list of column names.

In [ ]:
# export
@pandas_flavor.register_dataframe_method
def group_by_summary(self, by:T.Union[T.AnyStr, T.List], filter_cols:T.Union[None, T.List[str]]=None)-> pd.DataFrame:
    """
    
    """
    if not filter_cols:
        filter_cols = self.columns[~self.columns.isin([by] if isinstance(by, str) else by)]
    return self.groupby(by)[filter_cols].agg(['mean','median','sem','std','count'])

In [ ]:
df['cat_col'] = df[0] > 0.5

In [ ]:
df.group_by_summary('cat_col')

0                                             1            \
             mean    median       sem       std count      mean    median   
cat_col                                                                     
False    0.244111  0.226694  0.044509  0.154183    12  0.509380  0.525242   
True     0.754864  0.750890  0.057191  0.161760     8  0.217906  0.203976   

                                   ...         3                      \
              sem       std count  ...      mean    median       sem   
cat_col                            ...                                 
False    0.102191  0.353999    12  ...  0.502497  0.355096  0.077057   
True     0.047951  0.135626     8  ...  0.611216  0.681959  0.119260   

                                4                                      
              std count      mean    median       sem       std count  
cat_col                                                                
False    0.266935    12  0.523255  0.577585  0.080473  0.278766    12  
True     0.337319     8  0.487294  0.409391  0.130743  0.369796     8  

[2 rows x 25 columns]

In [ ]:
df.group_by_summary(by='cat_col', filter_cols=1)

,mean,median,sem,std,count
cat_col,,,,,
False,0.509380,0.525242,0.102191,0.353999,12
True,0.217906,0.203976,0.047951,0.135626,8


In [ ]:
df.group_by_summary(['cat_col'], filter_cols=1)

,mean,median,sem,std,count
cat_col,,,,,
False,0.509380,0.525242,0.102191,0.353999,12
True,0.217906,0.203976,0.047951,0.135626,8


### datetime_from_timestamp

Converts a pd.Series of integer timestamps to datetime format.

Options:

* `s`: pd.Series of timestamps
* `ts_type`: type of timestamp. 
  * `ms`: default, milliseconds
  * `ns`: nanoseconds
  * `s`: seconds

In [ ]:
# export
def datetime_from_timestamp(s:pd.Series, ts_type:str='ms') -> pd.Series:
    lookup = {
        'ms':1e3,
        's':1,
        'ns': 1e6
    }
    return s.map(lambda x: datetime.fromtimestamp(x/lookup[ts_type]))

In [ ]:
datetime_from_timestamp(pd.Series([1627417286117]))

0   2021-07-27 13:21:26.117
dtype: datetime64[ns]